In [19]:
import pandas as pd
import numpy as np
import datetime
import sqlite3
from pandas.tseries.holiday import USFederalHolidayCalendar as uscal

import requests
import json
import glob
import gzip

import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

import xgboost
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
%matplotlib inline

In [9]:
names = glob.glob('*.csv')
df_raw = pd.DataFrame()
for i in names:
    a = pd.read_csv(i)
    df_raw = pd.concat([a, df_mp])

In [12]:
df_mp = df_raw.reset_index()
df_mp['Timestamp'] = pd.to_datetime(df_mp['Timestamp'])
df_mp['date'] = df_mp['Timestamp'].dt.date

In [15]:
df_mp['sunrise'] = 0
df_mp['sunset'] = 0

def data_sunrise(date):
    url = 'https://api.sunrise-sunset.org/json?lat=40.82014&lng=-73.92255&date={}'.format(date)
    info = requests.get(url)
    new = json.loads(info.text)
    return new['results']['sunrise']

def data_sunset(date):
    url = 'https://api.sunrise-sunset.org/json?lat=37.97192&lng=-122.51619&date={}'.format(date)
    info = requests.get(url)
    new = json.loads(info.text)
    return new['results']['sunset']

for i in range(len(df_mp)):

    
    df_mp['sunrise'][i]= data_sunrise(df_mp['date'][i])
    df_mp['sunset'][i]= data_sunset(df_mp['date'][i])


C:\Users\szhang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\szhang\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\szhang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [17]:
df_mp['sunrise_est'] = pd.to_datetime(df_mp['sunrise'] - pd.Timedelta(hours=4)).dt.time
df_mp['sunset_est'] = pd.to_datetime(df_mp['sunset'] - pd.Timedelta(hours=4)).dt.time

C:\Users\szhang\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3296: FutureWarning: Passing timedelta64-dtype data is deprecated, will raise a TypeError in a future version
  exec(code_obj, self.user_global_ns, self.user_ns)


In [18]:
df_mp.to_csv('data_sunhour.csv')

In [137]:
df_mp = pd.read_csv('data_sunhour.csv')

In [138]:
df_mp = df_mp.rename(columns = {'655 Morris Ave Cellar Electric Meter Room meter electric EM-1':'655_Morris',
                       '2980 Park Ave Cellar Electric Meter Room meter electric EM-1':'2980_Park'})

In [139]:
min(df_mp.Timestamp), max(df_mp.Timestamp)

('2021-06-08 12:45:00', '2021-09-16 15:30:00')

In [140]:
# weather
weather_x = pd.read_csv('1276283e695618f359b6aad0226280e0.tar.gz',
                     compression='gzip', header=1)

In [141]:
weather = weather_x[['dt_iso',  'temp',
       'feels_like', 'pressure', 'humidity', 
        'wind_speed', 'wind_deg', 'rain_1h',
       'clouds_all', 'weather_main', 'weather_description']]
weather = weather.drop_duplicates(subset='dt_iso', keep='first')

In [142]:
weather.head(2)

,dt_iso,temp,feels_like,pressure,humidity,wind_speed,wind_deg,rain_1h,clouds_all,weather_main,weather_description
0,2006-01-01 00:00:00 +0000 UTC,1.01,-3.42,1009.0,88.0,4.6,360.0,0.19,90.0,Mist,mist
2,2006-01-01 01:00:00 +0000 UTC,0.56,-3.66,1009.0,88.0,4.1,360.0,NaN,90.0,Snow,light snow


In [167]:
weather['timestamp'] = pd.to_datetime(weather.dt_iso.str.split('+').str[0])
weather['timestamp'] = pd.to_datetime(weather['timestamp'] - pd.Timedelta(hours=4))
wed_range = weather[(weather['timestamp'] > pd.to_datetime('2021-06-08 11:00:00')) & (weather['timestamp'] < pd.to_datetime('2021-09-16 15:30:00'))].reset_index(drop=True)
wed_range['timestamp'] = pd.to_datetime(wed_range['timestamp'])

In [168]:
df_mp['Morris_u'] = df_mp['655_Morris'].str.replace('kWh', '').astype(float)
df_mp['Park_u'] = df_mp['2980_Park'].str.replace('kWh', '').astype(float)
df_mp['Timestamp'] = pd.to_datetime(df_mp['Timestamp'])
df_mp['date'] = df_mp['Timestamp'].dt.date
df_mp['hour'] = df_mp['Timestamp'].dt.hour
df_mp['hour'] = pd.to_timedelta(df_mp['hour'], unit='h')
df_mp['timestamp_hour'] = pd.to_datetime(df_mp['date']) + df_mp['hour']

In [169]:
df_mp.columns

Index(['Unnamed: 0', 'index', 'Timestamp', '655_Morris', '2980_Park', 'date',
       'sunrise', 'sunset', 'sunrise_est', 'sunset_est', 'Morris_u', 'Park_u',
       'hour', 'timestamp_hour'],
      dtype='object')

In [170]:
#put df into memory space as db
conn = sqlite3.connect(':memory:')
#write the tables
df_mp.to_sql('data', conn, index=False)

C:\Users\szhang\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\io\sql.py:460: UserWarning: the 'timedelta' type is not supported, and will be written as integer values (ns frequency) to the database.
  chunksize=chunksize, dtype=dtype, method=method)
C:\Users\szhang\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:2531: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype, method=method)


In [171]:
# using SQL to create new column faster
qry = '''
select Timestamp, date,
       sunrise_est, sunset_est, Morris_u, Park_u,
       hour, timestamp_hour,
CASE
when time(timestamp_hour) > time(sunrise_est) and time(timestamp_hour) < time(sunset_est) THEN "Yes"
else "No"
END as Sunlight
from data

'''
mp_hour = pd.read_sql_query(qry, conn)

In [190]:
mp_hour['timestamp_hour'] = pd.to_datetime(mp_hour['timestamp_hour'])
mp_hour_sum = pd.DataFrame(mp_hour.groupby(['timestamp_hour'])['Morris_u', 'Park_u'].sum()).reset_index()

In [192]:
hours_weather = pd.merge(mp_hour_sum, wed_range, 
        left_on='timestamp_hour', right_on='timestamp', how = 'left')

In [194]:
df_model = hours_weather[['timestamp_hour', 'Morris_u',
                          'Park_u',  'temp', 'pressure', 'humidity', 'wind_speed',
       'wind_deg', 'clouds_all', 'weather_main',
       'weather_description']].reset_index()

# Modeling

In [195]:
X.head()

,temp,pressure,humidity,hour,weekday
0,26.72,1008.0,52.0,00:00:00,2
1,NaN,NaN,NaN,00:00:00,2
2,NaN,NaN,NaN,00:00:00,2
3,NaN,NaN,NaN,00:00:00,2
4,26.47,1008.0,55.0,01:00:00,2


In [197]:
# Modeling - baseline
y = df_model['Morris_u']
X =  df_model[['temp', 'pressure', 'humidity']]

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                    test_size=0.25, random_state=123)
lr = LinearRegression()
lr_mod = lr.fit(X_train, y_train)
y_pred = lr_mod.predict(X_test)
y_predt = lr_mod.predict(X_train)
mean_absolute_error(y_train, y_predt),mean_absolute_error(y_test, y_pred)

(7.946487195093166, 7.495530301666456)